# Import libraries & dataframes

In [2]:
# import libraries
import pandas as pd
import numpy as np
import os

In [3]:
path = r'C:\Users\muril\Data Analysis Projects\07-2023 Instacart Basket Analysis\02 Data'

In [3]:
# Import dataframes
df_ords_prior = pd.read_csv(os.path.join(path, 'Original Data', 'order_products__prior.csv'), index_col=False)

In [4]:
df_prods = pd.read_csv(os.path.join(path,'Prepared Data','products_cleaned.csv'), index_col=0)

In [5]:
df_ords = pd.read_csv(os.path.join(path, 'Prepared Data', 'orders_cleaned.csv'), index_col=0)

In [6]:
cust = pd.read_csv(os.path.join(path, 'Prepared Data', 'customers_cleaned.csv'), index_col=0)

In [4]:
reg = pd.read_csv(os.path.join(path, 'Original Data', 'regions.csv'))

In [5]:
df = pd.read_csv(os.path.join(path, 'Prepared Data', 'instacart_wrangled.csv'))

MemoryError: Unable to allocate 3.38 GiB for an array with shape (14, 32433030) and data type int64

# 4. Merge dataframes

## 4.1 Merge orders and orders_prior

In [7]:
# Check df_ords_prior shape
df_ords_prior.shape

(32434489, 4)

In [8]:
# Check df_ords shape
df_ords.shape

(3421083, 6)

In [9]:
# Merge dataframes
df_merged_large = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True)

In [10]:
df_merged_large.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,196,1,0,both
1,2539329,1,1,2,8,NaN,14084,2,0,both
2,2539329,1,1,2,8,NaN,12427,3,0,both
3,2539329,1,1,2,8,NaN,26088,4,0,both
4,2539329,1,1,2,8,NaN,26405,5,0,both


In [11]:
# Check merge
df_merged_large['_merge'].value_counts()

both          32434489
left_only            0
right_only           0
Name: _merge, dtype: int64

We cannot conclude here that every observation was merged between the two dataframes, despite the 0 counts for left and right only. Since it was an inner join, by definition the left and right counts would have to be 0. Merging by outer would show the true number of mismatching records.

In [12]:
# Check dimensions of merged dataframe
df_merged_large.shape

(32434489, 10)

In [13]:
# drop _merge column
df_merged_large = df_merged_large.drop('_merge', axis=1)
df_merged_large.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,1,2,8,NaN,196,1,0
1,2539329,1,1,2,8,NaN,14084,2,0
2,2539329,1,1,2,8,NaN,12427,3,0
3,2539329,1,1,2,8,NaN,26088,4,0
4,2539329,1,1,2,8,NaN,26405,5,0


## 4.2 Merge products

In [14]:
# check dimensions
df_prods.shape

(49688, 5)

In [15]:
df_merged_large.shape

(32434489, 9)

In [16]:
# merge dataframes
df_merged = df_merged_large.merge(df_prods, on='product_id', how='left'

In [17]:
df_merged.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0


In [18]:
df_merged.isnull().sum()

order_id                        0
user_id                         0
order_number                    0
order_day_of_week               0
order_hour_of_day               0
days_since_prior_order    2077979
product_id                      0
add_to_cart_order               0
reordered                       0
product_name                28171
aisle_id                        0
department_id                   0
prices                          0
dtype: int64

There are nulls in the column that we expect there to be nulls, as in the previous scripts.

In [19]:
df_merged.shape

(32433030, 13)

## 4.3 Merge customers

In [20]:
# list columns of dataframes
df_merged.columns

Index(['order_id', 'user_id', 'order_number', 'order_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices'],
      dtype='object')

In [21]:
cust.columns

Index(['user_id', 'sex', 'state', 'age', 'date_joined', 'n_dependents',
       'family_status', 'income'],
      dtype='object')

In [22]:
del df_ords
del df_prods
del df_ords_prior

In [23]:
# merge customers with df_merged
df_merged = df_merged.merge(cust, on='user_id', how='left')

In [26]:
# check for nulls 
df_merged.isnull().sum()

order_id                        0
user_id                         0
order_number                    0
order_day_of_week               0
order_hour_of_day               0
days_since_prior_order    2077979
product_id                      0
add_to_cart_order               0
reordered                       0
product_name                28171
aisle_id                        0
department_id                   0
prices                          0
sex                             0
state                           0
age                             0
date_joined                     0
n_dependents                    0
family_status                   0
income                          0
dtype: int64

## 4.4 Merge regions

In [ ]:
# merge regions
df.merge(regions['state','region'], on='state', how='left')

# Export dataframe

In [29]:
# Export data to csv
df_merged.to_csv(os.path.join(path, 'Prepared Data', 'instacart_wrangled.csv'))

In [30]:
# Export data to pkl
df_merged.to_pickle(os.path.join(path,'Prepared Data', 'instacart_wrangled.pkl'))